In [1]:
import pandas as pd
df = pd.read_parquet('df.parquet')

In [2]:
horizon_len = 7
patch_len = 32
patch_num = 20
batch_size = 32
hidden_size = 128
channels = ['Close', 'High','Low',	'Open',	'Volume']

In [3]:
from deepdow.utils import raw_to_Xy


n_timesteps = len(df)  # 20
n_channels = len(channels)  # 2
#n_assets = len(df.columns.levels[0])  # 2

lookback, gap, horizon = patch_len*patch_num, 2, horizon_len

X, timestamps, y, asset_names, indicators = raw_to_Xy(df,
                                                      lookback=lookback,
                                                      gap=gap,
                                                      freq="B",
                                                      horizon=horizon,
                                                      included_indicators=channels)

n_samples =  n_timesteps - lookback - horizon - gap + 1  # 10
n_assets = len(asset_names)


In [4]:
X.shape

(4309, 5, 640, 30)

In [5]:
timestamps

DatetimeIndex(['2008-09-08', '2008-09-09', '2008-09-10', '2008-09-11',
               '2008-09-12', '2008-09-15', '2008-09-16', '2008-09-17',
               '2008-09-18', '2008-09-19',
               ...
               '2025-02-28', '2025-03-03', '2025-03-04', '2025-03-05',
               '2025-03-06', '2025-03-07', '2025-03-10', '2025-03-11',
               '2025-03-12', '2025-03-13'],
              dtype='datetime64[ns]', length=4309, freq='B')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

#model.config.output_attentions = True  # 返回注意力权重
model.config.output_hidden_states = True  # 返回隐藏状态（可选）


In [6]:
split_ix = int(n_samples * 0.975)
indices_train = list(range(split_ix))
indices_test = list(range(split_ix + horizon, n_samples))

print('Train range: {}:{}\nTest range: {}:{}'.format(indices_train[0], indices_train[-1],
                                                     indices_test[0], indices_test[-1]))

Train range: 0:4028
Test range: 4036:4132


In [7]:
from deepdow.data import InRAMDataset, RigidDataLoader, prepare_standard_scaler, Scale
means, stds = prepare_standard_scaler(X, indices=indices_train)
print('mean: {}, std: {}'.format(means, stds))

mean: [ 0.00043403  0.0004326   0.00043554  0.00043343 -0.00022286], std: [0.01764646 0.01503178 0.01667324 0.01733977 0.3423143 ]


In [8]:

dataset = InRAMDataset(X, y, transform=Scale(means, stds))

In [9]:
dataloader_train = RigidDataLoader(dataset,
                                   indices=indices_train,
                                   batch_size=batch_size)

dataloader_test = RigidDataLoader(dataset,
                                  indices=indices_test,
                                  batch_size=batch_size)

In [10]:
tmp = next(iter(dataloader_train))

In [22]:
len(X)

4309

In [12]:
timestamps[tmp[2]]

DatetimeIndex(['2012-11-26', '2012-04-26', '2012-06-04', '2021-10-20',
               '2023-05-10', '2022-01-20', '2012-05-17', '2013-11-29',
               '2009-03-03', '2010-01-04', '2021-04-09', '2020-10-27',
               '2021-08-25', '2013-05-15', '2012-08-17', '2023-11-17',
               '2013-07-11', '2014-04-25', '2015-09-28', '2018-08-03',
               '2017-10-03', '2019-06-27', '2022-12-29', '2017-06-21',
               '2024-01-18', '2015-06-01', '2008-09-09', '2023-08-07',
               '2020-06-17', '2012-07-17', '2011-03-31', '2009-05-04'],
              dtype='datetime64[ns]', freq=None)